In [1]:
import GetOldTweets3 as got
import datetime
import pandas as pd
import re
import yfinance as yf

In [2]:
def get_Tweets(query,start,end,maxTweets=150):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query).setTopTweets(True).setSince(start).setUntil(end).setMaxTweets(maxTweets)
    tweets = got.manager.TweetManager().getTweets(tweetCriteria)
    return tweets

In [9]:
df = pd.DataFrame(columns=['Date','Tweets'])
date = datetime.datetime(2020,6,29)
tweets = list()
while date != datetime.datetime(2020,7,3):
    startDate = str(date).split()[0]
    endDate = str(date + datetime.timedelta(days=1)).split()[0]
    
    tweets.append(get_Tweets('AAPL',startDate,endDate))
    date += datetime.timedelta(days=1)

In [10]:
for tweet in tweets:
    if type(tweet) == type(tweets):
        for t in tweet:
            df = df.append(pd.Series([t.date,t.text], index=df.columns),ignore_index=True)
    else:
        df = df.append(pd.Series([tweet.date,tweet.text], index=df.columns),ignore_index=True)  

In [11]:
df

,Date,Tweets
0,2020-06-29 23:26:42+00:00,$BA - Back-testing upwards trendline. It can p...
1,2020-06-29 20:07:49+00:00,CHECK OUT @STARINKSIGNALS #options #optiontrad...
2,2020-06-29 19:07:30+00:00,$GOOG $AAPL $SPY $AMZN $TDOC $TSLA $COST $BA $...
3,2020-06-29 17:29:54+00:00,"$AAPL - if the pattern continues, should be se..."
4,2020-06-29 16:58:10+00:00,$FB nice recovery off cloud high and recovered...
...,...,...
595,2020-07-02 01:39:14+00:00,People are loaded to the gills with the same s...
596,2020-07-02 01:16:58+00:00,July 2 Plan *Click on Picture to Expand* Pleas...
597,2020-07-02 01:08:11+00:00,I have a dispersion question I need help with....
598,2020-07-02 01:06:32+00:00,$GLD Bullish #OptionsAction!! $work $docu $okt...


In [12]:
i=0
for tweet in df.Tweets:
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    tweet = re.sub('@[^\s]+',' ',tweet)
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = tweet.strip('\'"')
    tweet = re.sub('[^a-zA-Z\s]*', '', tweet)
    df.at[i,'Tweets'] = tweet
    i+=1

In [13]:
df.drop_duplicates(subset=['Date','Tweets'],keep='first',inplace=True)
df['Date'] = df['Date'].apply(lambda x: str(x).split()[0])

In [8]:
#df.to_csv('DayWise.csv')

In [14]:
combine = lambda tweet:",".join(tweet)
grouped = df.groupby("Date").agg({'Tweets':combine},inplace=True).reset_index()

grouped.sort_values(by=['Date'],inplace=True)
grouped

,Date,Tweets
0,2020-06-29,ba backtesting upwards trendline it can possi...
1,2020-06-30,here is my semiannual report for the past mon...
2,2020-07-01,follow for optionsignals optiontrading will go...
3,2020-07-02,msft outperforming aapl fb nflf today ready f...


In [30]:
#grouped.to_csv('GetOldTweets.csv')
grouped.to_csv('Test28-30.csv')

In [11]:
df1 = pd.read_csv('Test28-30.csv',usecols=['Date','Tweets'])
df1

,Date,Tweets
0,2020-06-28,that feeling when aapl falls apart and you see...
1,2020-06-29,ba backtesting upwards trendline it can possi...
2,2020-06-30,here is my semiannual report for the past mon...


In [31]:
merged = pd.concat([grouped,df1])
merged.sort_values(by=['Date'],inplace=True)
merged = merged.reset_index()
merged = merged.drop(['index'],axis=1)
merged

,Date,Tweets
0,2020-01-01,market driver 2019 aapl had highest approx tur...
1,2020-01-02,2020 is already a memetic year tsla is over 42...
2,2020-01-03,the sectorselector for 010320 is posted techno...
3,2020-01-04,wild that apple aapl the biggest company in th...
4,2020-01-05,do you want to learn to trade stocks using bac...
...,...,...
172,2020-06-22,top increases in users holding for stocks held...
173,2020-06-23,aapl apple appeared to have brokeout but weak ...
174,2020-06-24,ctrm come on shorts u guys r weak as fuck brin...
175,2020-06-25,my advice to all robinhooders get webull and s...


In [32]:
#merged.to_csv('FINAL1-6.csv')

In [18]:
data = yf.download("AAPL", start="2020-06-28", end="2020-07-01")

[*********************100%***********************]  1 of 1 completed


In [19]:
data = data.drop(columns=['Open','High','Low','Close','Volume'],axis=1)

In [20]:
stockPrices = data.rename(columns={'Adj Close':'AdjClose'})
stockPrices = stockPrices.reset_index()
stockPrices['Date'] = stockPrices['Date'].astype(str)
stockPrices.set_index('Date',inplace=True)
stockPrices.head()

,AdjClose
Date,
2020-06-29,361.779999
2020-06-30,364.799988


In [21]:
#stockPrices.to_csv('testY.csv')

In [27]:
tweets = pd.read_csv('FINALMERGED.csv',usecols=['Date','Tweets'])
tweets['Date'] = tweets['Date'].astype(str)

In [45]:
merged = pd.DataFrame(tweets['Date'])


,Date
0,2020-04-20
1,2020-04-21
2,2020-04-22
3,2020-04-23
4,2020-04-24
...,...
62,2020-06-22
63,2020-06-23
64,2020-06-24
65,2020-06-25


'2020-04-20'